In [6]:
from selectorlib import Extractor
import requests 
import json 
from time import sleep
import pandas as pd
import numpy as np
import time

# Make a dataframe to store the information scrapped from Amazon.
pricelist = pd.DataFrame(columns=['Name','Price','GPU'])
 

# Function to scrap the information from Amazon.
def x(GPU):
    # GPU is a name of GPU which we want to search on Amazon
    # Create an Extractor by reading from the YAML file
    e = Extractor.from_yaml_file('search_results.yml')

    def scrape(url):  
        
        ### If there is an issue with scrapping, try changing the agent.
        # Following is the list of agents you can try
        
        #["Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",\
        #        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1500.72 Safari/537.36",\
        #        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10) AppleWebKit/600.1.25 (KHTML, like Gecko) Version/8.0 Safari/600.1.25",\
        #        "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:33.0) Gecko/20100101 Firefox/33.0",\
        #        "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",\
        #        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",\
        #        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/600.1.17 (KHTML, like Gecko) Version/7.1 Safari/537.85.10",\
        #        "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko",\
        #        "Mozilla/5.0 (Windows NT 6.3; WOW64; rv:33.0) Gecko/20100101 Firefox/33.0",\
        #        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.104 Safari/537.36"\
        #        ]
        
        headers = {
            'dnt': '1',
            'upgrade-insecure-requests': '1',
            'user-agent': "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'sec-fetch-site': 'same-origin',
            'sec-fetch-mode': 'navigate',
            'sec-fetch-user': '?1',
            'sec-fetch-dest': 'document',
            'referer': 'https://www.amazon.com/',
            'accept-language': 'en-US;q=0.9,en;q=0.8',
        }
        # Download the page using requests
        print("Downloading %s"%url)
        r = requests.get(url, headers=headers)
        # Simple check to check if page was blocked (Usually 503)
        if r.status_code > 500:
            if "To discuss automated access to Amazon data please contact" in r.text:
                print("Page %s was blocked by Amazon. Please try using better proxies\n"%url)
            else:
                print("Page %s must have been blocked by Amazon as the status code was %d"%(url,r.status_code))
            return None
        # Pass the HTML of the page and create 
        return e.extract(r.text)

    # product_data = []
    
    with open("search_results_urls.txt",'r') as urllist, open('search_results_output.jsonl','w') as outfile:        
        for url in urllist.read().splitlines():
            data = scrape(url) 
                    
            if data:
                for product in data['products']:
                    leng = len(GPU.split(" "))
                    count=0
                    for gpu in GPU.split(" "):
                        
                        if gpu in product['title']:
                            count+=1
                        else:
                            pass 
                        try:
                            if len(GPU.split(" "))==count:
                                pricelist.loc[len(pricelist)] = [product['title'],float(product['price'].replace(",",'').replace("$",'')),GPU]
                        except:
                            pass    

                    product['search_url'] = url
                    print("Saving Product: %s"%product['title'])
                    json.dump(product,outfile)
                    outfile.write("\n")
                    # sleep(5)

                    
## Knapsack problem solver
def knapsack_unbounded_dp(items, C):
    # order by max value per item size
    items = sorted(items, key=lambda item: item[VALUE]/float(item[SIZE]), reverse=True)
 
    # Sack keeps track of max value so far as well as the count of each item in the sack
    sack = [(0, [0 for i in items]) for i in range(0, C+1)]   # value, [item counts]
 
    for i,item in enumerate(items):
        name, size, value = item
        for c in range(size, C+1):
            sackwithout = sack[c-size]  # previous max sack to try adding this item to
            trial = sackwithout[0] + value
            used = sackwithout[1][i]
            if sack[c][0] < trial:
                # old max sack with this added item is better
                sack[c] = (trial, sackwithout[1][:])
                sack[c][1][i] +=1   # use one more
 
    value, bagged = sack[C]
    numbagged = sum(bagged)
    size = sum(items[i][1]*n for i,n in enumerate(bagged))
    # convert to (iten, count) pairs) in name order
    bagged = sorted((items[i][NAME], n) for i,n in enumerate(bagged) if n)
 
    return value, size, numbagged, bagged

In [7]:
# Read GPU file which we obtained from Nicehash.
# Read the paper for the link.

gpu = pd.read_csv("GPU.csv")

gpu['profit'] = gpu['Net Profit'].str.split(" ").str[0]
gpu['profit'] =gpu['profit'].astype(float)

for gp in gpu['Model'].values:
    # Make URL for each GPU
    f = open("search_results_urls.txt","w")
    f.write("https://www.amazon.com/s?k="+str(gp))
    f.close()
    # Call scrapper
    x(gp)

# Save price information.
pricelist.to_csv("prices.csv")

In [8]:
# Read GPU file which we obtained from Nicehash.
gpu = pd.read_csv("GPU.csv")
# Read Prices file which we obtained from Amazon.
prices  = pd.read_csv("prices.csv",index_col=0)

gpu['profit'] = gpu['Net Profit'].str.split(" ").str[0]
gpu['profit'] =gpu['profit'].astype(float)

asdprices = prices.sort_values(by = 'Price',ascending = True)
decprices = prices.sort_values(by = 'Price',ascending = False)
low = asdprices.drop_duplicates(subset ="GPU",keep = 'first')
high = decprices.drop_duplicates(subset ="GPU",keep = 'first')

gpu = gpu[gpu['Model'].isin(list(low['GPU'].values))]
gpu = gpu[['Model','profit']]

gpu = gpu.sort_values(by = 'Model',ascending = True)
low = low.sort_values(by = 'GPU',ascending = True)
high = high.sort_values(by = 'GPU',ascending = True)
low['profit'] = gpu['profit'].values
high['profit'] = gpu['profit'].values
low = low[['GPU','Price','profit']]
high = high[['GPU','Price','profit']]
low['Recoverytimeindays'] = low['Price']/low['profit']
high['Recoverytimeindays'] = high['Price']/high['profit']

# Get low and high price for both products if there is a duplication.
low = low.sort_values(by = 'Recoverytimeindays',ascending = True)
high = high.sort_values(by = 'Recoverytimeindays',ascending = True)
print(low)
print(high)


from operator import itemgetter as iget
from itertools import product
from random import shuffle
low = low[['GPU','Price','profit']]
high = high[['GPU','Price','profit']]
low['Price'] = low['Price'].astype(int)
high['Price'] = high['Price'].astype(int)

NAME, SIZE, VALUE = range(3)
itemslow = low.values
itemshigh = high.values

capacity = 5000

# Simple Case
print("Money: ",capacity," :", knapsack_unbounded_dp(high.values,capacity)) 

                       GPU    Price  profit  Recoverytimeindays
5           NVIDIA TITAN V  1099.99    3.35          328.355224
28      NVIDIA RTX 3060 Ti   899.99    2.58          348.833333
32         NVIDIA RTX 3060   859.99    2.12          405.655660
63   NVIDIA GTX 1660 Super   586.15    1.31          447.442748
36     AMD RX 6700 XT 12GB   893.38    1.99          448.934673
66         AMD RX 6600 8GB   536.61    1.19          450.932773
53         NVIDIA RTX 2060   631.59    1.40          451.135714
3          NVIDIA RTX 3080  1887.00    4.11          459.124088
55      AMD RX 6600 XT 8GB   629.99    1.36          463.227941
19        AMD RX 6800 16GB  1259.00    2.71          464.575646
26         NVIDIA RTX 3070  1299.99    2.63          494.292776
47      NVIDIA GTX 1080 Ti   899.99    1.82          494.500000
72         NVIDIA GTX 1650   337.22    0.68          495.911765
46   NVIDIA RTX 2070 Super   949.99    1.86          510.747312
48         NVIDIA RTX 2070   949.99    1

In [9]:
# Variation in Money
for loop in range(1000,11000,1000):
    print("Money: ",loop,",","Recovery time: ",int(loop/knapsack_unbounded_dp(high.values, loop)[0]),knapsack_unbounded_dp(high.values, loop)) 

Money:  1000 , Recovery time:  502 (1.9900000000000002, 1000, 2, [('NVIDIA GTX 1650', 1), ('NVIDIA GTX 1660 Super', 1)])
Money:  2000 , Recovery time:  476 (4.199999999999999, 1893, 3, [('NVIDIA RTX 2060', 3)])
Money:  3000 , Recovery time:  451 (6.640000000000001, 2975, 5, [('NVIDIA GTX 1660 Super', 4), ('NVIDIA RTX 2060', 1)])
Money:  4000 , Recovery time:  458 (8.72, 3979, 7, [('AMD RX 580 4GB', 1), ('NVIDIA GTX 1660 Super', 3), ('NVIDIA RTX 2060', 3)])
Money:  5000 , Recovery time:  453 (11.020000000000001, 4958, 8, [('NVIDIA GTX 1660 Super', 2), ('NVIDIA RTX 2060', 6)])
Money:  6000 , Recovery time:  448 (13.370000000000003, 5995, 10, [('NVIDIA GTX 1660 Super', 7), ('NVIDIA RTX 2060', 3)])
Money:  7000 , Recovery time:  453 (15.450000000000003, 6999, 12, [('AMD RX 580 4GB', 1), ('NVIDIA GTX 1660 Super', 6), ('NVIDIA RTX 2060', 5)])
Money:  8000 , Recovery time:  450 (17.75, 7978, 13, [('NVIDIA GTX 1660 Super', 5), ('NVIDIA RTX 2060', 8)])
Money:  9000 , Recovery time:  449 (20.01,

In [15]:
# Variation in time period
# We considered a constant decrease of 0.1 in profit.
# We considered high values for further analysis.

possiblesolutions = {}

# Time step
timesteps = 10
temphigh = high.copy()
for loop in range(0,timesteps):
    print("\n")
    print("Decreasing profit values for all GPUs: ",loop)
    print(knapsack_unbounded_dp(itemshigh,capacity))
    possiblesolutions[loop]= knapsack_unbounded_dp(itemshigh,capacity)
    temphigh['profit']= temphigh['profit'] - 0.1
    itemshigh = temphigh.values
print("\n")
tableoftimestep = pd.DataFrame()
for key in possiblesolutions:
    tempprofit = 0
    tempprofitlist = []
    for time in range(0,timesteps): 
        temp = 0
        for GPU in possiblesolutions[key][3]:
            temp = temp+ high[high['GPU']==GPU[0]]['profit'].values[0]*GPU[1]-0.1*time*GPU[1]
        tempprofitlist.append(temp)    
    tableoftimestep["time_step"+str(key)] = tempprofitlist
print(tableoftimestep)
sum_column = tableoftimestep.sum(axis=0)
print("From following choose the max time step")
print("For us it was timestep 2,3,4,5")
print(sum_column)




Decreasing profit values for all GPUs:  0
(7.339999999999999, 4913, 3, [('NVIDIA RTX 3060', 1), ('NVIDIA RTX 3080', 2)])


Decreasing profit values for all GPUs:  1
(10.530000000000001, 4991, 4, [('NVIDIA GTX 1660 Super', 1), ('NVIDIA RTX 2060', 1), ('NVIDIA RTX 3080', 2)])


Decreasing profit values for all GPUs:  2
(10.13, 4991, 4, [('NVIDIA GTX 1660 Super', 1), ('NVIDIA RTX 2060', 1), ('NVIDIA RTX 3080', 2)])


Decreasing profit values for all GPUs:  3
(9.73, 4991, 4, [('NVIDIA GTX 1660 Super', 1), ('NVIDIA RTX 2060', 1), ('NVIDIA RTX 3080', 2)])


Decreasing profit values for all GPUs:  4
(9.33, 4991, 4, [('NVIDIA GTX 1660 Super', 1), ('NVIDIA RTX 2060', 1), ('NVIDIA RTX 3080', 2)])


Decreasing profit values for all GPUs:  5
(8.930000000000001, 4991, 4, [('NVIDIA GTX 1660 Super', 1), ('NVIDIA RTX 2060', 1), ('NVIDIA RTX 3080', 2)])


Decreasing profit values for all GPUs:  6
(8.54, 4913, 3, [('NVIDIA RTX 3060', 1), ('NVIDIA RTX 3080', 2)])


Decreasing profit values for all GPUs